In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

# Load Data

In [ ]:
cleaned_data_path = pathlib.Path("../data/interim/ctg_cleaned_manually.xlsx")
data = pd.read_excel(cleaned_data_path, header=0)
x_raw = data.loc[:, "Min":"SUSP"].to_numpy()
y_raw_class = data.loc[:, "CLASS"].to_numpy()
y_raw_nsp = data.loc[:, "NSP"].to_numpy()

## One Hot Encoding for Categorical Crossentropy Loss

In [ ]:
y_class_one_hot = tf.one_hot(indices=y_raw_class, depth=max(y_raw_class)).numpy()
y_nsp_one_hot = tf.one_hot(indices=y_raw_nsp, depth=max(y_raw_nsp)).numpy()

### Split into Train and Test
90% of the Data is Training Data

In [ ]:
x_train = x_raw[:1913]
x_test = x_raw[1913:]
y_train = y_nsp_one_hot[:1913]
y_test = y_nsp_one_hot[1913:]

## Baseline Model Creation

In [ ]:
BATCH_SIZE = 4
EPOCHS = 15

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=19, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=128, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

In [ ]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics="categorical_accuracy"
)

In [ ]:
model.build(input_shape=(BATCH_SIZE, x_raw.shape[1]))

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    x=x_train, y=y_train, batch_size=BATCH_SIZE, validation_split=0.2, epochs=EPOCHS
)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

In [ ]:
plt.plot(history.history["categorical_accuracy"])
plt.plot(history.history["val_categorical_accuracy"])

In [ ]:
model.evaluate(x=x_test, y=y_test)

In [ ]:
model.save("../models/baseline_keras")

In [ ]:
%%timeit
model.predict(x_test[0:1])